In [9]:
import pandas as pd 
import json
import joblib

In [10]:
### Load Week Update Data
data = pd.read_csv('CFB_Pred_Week_4.csv')
data.head()

,away_team,home_team,game_id,season,week,neutral_site,conference_game,spread,home_favored,home_covered,...,away_total_receiving_ppa,away_total_rushing_ppa,away_percent_ppa,away_percent_passing_ppa,away_percent_receiving_ppa,away_percent_rushing_ppa,away_usage,away_passing_usage,away_receiving_usage,away_rushing_usage
0,Arizona State,Baylor,401756896,NaN,4,NaN,NaN,2.500000,1,NaN,...,181.9,57.2,0.695,0.903,0.760,0.389,0.687,0.987,0.691,0.469
1,Arkansas,Memphis,401752708,NaN,4,NaN,NaN,7.500000,0,NaN,...,13.6,58.6,0.361,0.848,0.048,0.470,0.553,0.926,0.091,0.492
2,Arkansas State,Kennesaw State,401757261,NaN,4,NaN,NaN,5.000000,0,NaN,...,146.2,25.0,0.586,0.896,0.534,0.354,0.701,0.945,0.443,0.596
3,Auburn,Oklahoma,401752714,NaN,4,NaN,NaN,7.166667,1,NaN,...,95.0,2.6,0.218,0.000,0.383,0.037,0.190,0.000,0.426,0.232
4,Ball State,UConn,401757169,NaN,4,NaN,NaN,21.166667,1,NaN,...,34.2,-6.5,0.077,-0.007,0.146,-0.207,0.109,0.024,0.161,0.179


In [11]:
with open("selected_features_best_model_20250915.json", "r") as f:
    pred_features = json.load(f)
pred_features

['home_prev_week_Offense_First_Down_Success_Rate',
 'away_possession_time_avg3',
 'away_interception_tds_avg_all',
 'home_passes_deflected_allowed_avg_all',
 'away_prev_week_punt_returns_allowed',
 'away_interception_yards_allowed_avg3',
 'away_fourth_down_attempts_allowed_avg_all',
 'away_total_plays_against_avg3',
 'home_Defense_First_Down_Pass_Success_Rate_avg3',
 'away_Offense_First_Down_Run_Success_Rate_avg_all',
 'away_prev_week_total_yards_allowed',
 'away_penalty_yard_margin_avg3',
 'away_rushing_attempts_allowed_avg3',
 'home_Offense_First_Down_Success_avg3',
 'away_prev_week_point_differential',
 'away_kick_return_yards_avg3',
 'home_net_passing_yards_avg3',
 'away_prev_week_Defense_Total_Pass_Plays',
 'away_tackles_avg_all',
 'away_prev_week_interception_yards',
 'away_prev_week_Offense_Scoring_Drive_Percentage',
 'away_Defense_Total_Run_Plays_avg3',
 'away_prev_week_Defense_first_down_pass_rate',
 'away_sacks_avg_all',
 'home_prev_week_total_plays_against',
 'away_rushing_u

In [12]:
pred_data = data[pred_features]
pred_data

,home_prev_week_Offense_First_Down_Success_Rate,away_possession_time_avg3,away_interception_tds_avg_all,home_passes_deflected_allowed_avg_all,away_prev_week_punt_returns_allowed,away_interception_yards_allowed_avg3,away_fourth_down_attempts_allowed_avg_all,away_total_plays_against_avg3,home_Defense_First_Down_Pass_Success_Rate_avg3,away_Offense_First_Down_Run_Success_Rate_avg_all,...,away_Scaled_Talent,away_prev_week_sack_percentage,away_rushing_yards_allowed_avg3,spread,away_punt_returns_allowed_avg3,home_Offense_EPA_per_Play_avg_all,home_Defense_First_Down_Run_Success_Rate_avg3,away_tackles_for_loss_allowed_avg3,away_Total_Offense_EPA_Run_avg3,home_sacks_allowed_avg3
0,0.444444,0.00,0.000000,5.0,2,0.000000,3.500000,0,0.000000,0.565527,...,0.555900,0.090909,0,2.500000,0.000000,0.186450,0.000000,0.000000,0.000000,0
1,0.482759,0.00,0.000000,1.0,1,0.000000,1.000000,0,0.000000,0.471726,...,0.776179,0.111111,0,7.500000,0.000000,0.368633,0.000000,0.000000,0.000000,0
2,0.190476,0.00,0.000000,3.0,1,0.000000,1.500000,0,0.000000,0.448916,...,-0.394745,0.000000,0,5.000000,0.000000,-0.163816,0.000000,0.000000,0.000000,0
3,0.437500,0.00,0.000000,2.0,0,0.000000,3.500000,0,0.000000,0.459936,...,1.545733,0.375000,0,7.166667,0.000000,0.172119,0.000000,0.000000,0.000000,0
4,0.517241,0.00,0.000000,5.5,4,0.000000,1.500000,0,0.000000,0.200000,...,-1.014004,0.151515,0,21.166667,0.000000,0.220705,0.000000,0.000000,0.000000,0
5,0.535714,0.00,0.000000,1.5,2,0.000000,2.000000,0,0.000000,0.330357,...,-0.268143,0.090909,0,15.000000,0.000000,0.273836,0.000000,0.000000,0.000000,0
6,0.320000,0.00,0.500000,3.5,1,0.000000,1.500000,0,0.000000,0.478022,...,-2.644143,0.166667,0,5.666667,0.000000,0.046442,0.000000,0.000000,0.000000,0
7,0.705882,0.00,0.000000,3.5,1,0.000000,1.000000,0,0.000000,0.620536,...,1.589247,0.027778,0,9.500000,0.000000,0.209805,0.000000,0.000000,0.000000,0
8,0.406250,29.15,0.333333,4.0,1,10.333333,1.666667,63,0.504873,0.392593,...,-0.423345,0.040816,149,2.333333,0.666667,-0.087745,0.278186,5.666667,9.302907,4
9,0.555556,0.00,0.000000,1.5,2,0.000000,1.000000,0,0.000000,0.359211,...,-0.261106,0.050000,0,28.333333,0.000000,0.503923,0.000000,0.000000,0.000000,0


In [14]:
### Load Model
import xgboost
model = joblib.load("best_xgb_model_20250915.pkl")

In [28]:
import numpy as np
X_pred = pred_data[model.get_booster().feature_names]

cover_pred = model.predict(X_pred)
cover_prob = model.predict_proba(X_pred)[:, 1]

data['Cover_Prob'] = cover_prob
data['Cover_Prediction'] = np.where(cover_prob >= 0.6, 1, 0)
data

,away_team,home_team,game_id,season,week,neutral_site,conference_game,spread,home_favored,home_covered,...,away_percent_ppa,away_percent_passing_ppa,away_percent_receiving_ppa,away_percent_rushing_ppa,away_usage,away_passing_usage,away_receiving_usage,away_rushing_usage,Cover_Prediction,Cover_Prob
0,Arizona State,Baylor,401756896,NaN,4,NaN,NaN,2.500000,1,NaN,...,0.695,0.903,0.760,0.389,0.687,0.987,0.691,0.469,1,0.636509
1,Arkansas,Memphis,401752708,NaN,4,NaN,NaN,7.500000,0,NaN,...,0.361,0.848,0.048,0.470,0.553,0.926,0.091,0.492,0,0.582800
2,Arkansas State,Kennesaw State,401757261,NaN,4,NaN,NaN,5.000000,0,NaN,...,0.586,0.896,0.534,0.354,0.701,0.945,0.443,0.596,0,0.439052
3,Auburn,Oklahoma,401752714,NaN,4,NaN,NaN,7.166667,1,NaN,...,0.218,0.000,0.383,0.037,0.190,0.000,0.426,0.232,0,0.372278
4,Ball State,UConn,401757169,NaN,4,NaN,NaN,21.166667,1,NaN,...,0.077,-0.007,0.146,-0.207,0.109,0.024,0.161,0.179,0,0.306685
5,Coastal Carolina,South Alabama,401761611,NaN,4,NaN,NaN,15.000000,1,NaN,...,0.969,0.840,1.000,1.025,0.933,0.842,1.000,0.978,0,0.262228
6,Delaware,Florida International,401757259,NaN,4,NaN,NaN,5.666667,1,NaN,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0.339734
7,Florida,Miami,401752709,NaN,4,NaN,NaN,9.500000,1,NaN,...,0.550,0.730,0.371,0.716,0.651,0.728,0.434,0.725,1,0.637642
8,Fresno State,Hawai'i,401760374,NaN,4,NaN,NaN,2.333333,0,NaN,...,0.172,0.037,0.138,1.087,0.300,0.026,0.330,0.635,1,0.620564
9,Georgia State,Vanderbilt,401752716,NaN,4,NaN,NaN,28.333333,1,NaN,...,0.685,0.649,0.708,0.607,0.741,0.655,0.697,0.903,0,0.463957


In [30]:
predictions = data[['home_team', 'away_team','spread','home_favored' ,'Cover_Prediction', 'Cover_Prob']].sort_values(by='Cover_Prob', ascending=False)

In [34]:
predictions[predictions['Cover_Prediction'] == 1].drop(columns=['spread','home_favored']).reset_index(drop=True).rename(columns={'Cover_Prob':'Home Team Cover Probability', 'Cover_Prediction':'Home Team Cover Prediction'})

,home_team,away_team,Home Team Cover Prediction,Home Team Cover Probability
0,Ole Miss,Tulane,1,0.753095
1,Colorado,Wyoming,1,0.713863
2,Indiana,Illinois,1,0.684590
3,Oregon,Oregon State,1,0.660317
4,Miami,Florida,1,0.637642
5,Baylor,Arizona State,1,0.636509
6,Hawai'i,Fresno State,1,0.620564
7,Duke,NC State,1,0.613516
8,Tennessee,UAB,1,0.606965
